## Imports

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
from fastaidev.tabular import *
from sklearn.model_selection import StratifiedKFold

## Paths

In [3]:
PATH = Config.data_path()/'competitions/elo-merchant-category-recommendation/'

In [4]:
train_df = pd.read_pickle(PATH/'joined')

In [5]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
feature_1,201917.0,3.105311e+00,1.186160e+00,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,5.000000e+00
feature_2,201917.0,1.745410e+00,7.513617e-01,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00
feature_3,201917.0,5.655690e-01,4.956833e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
target,201917.0,-3.936363e-01,3.850500e+00,-3.321928e+01,-8.831102e-01,-2.343689e-02,7.654530e-01,1.796507e+01
first_active_monthYear,201917.0,2.016509e+03,7.881988e-01,2.011000e+03,2.016000e+03,2.017000e+03,2.017000e+03,2.018000e+03
first_active_monthMonth,201917.0,7.378745e+00,3.340718e+00,1.000000e+00,5.000000e+00,8.000000e+00,1.000000e+01,1.200000e+01
first_active_monthWeek,201917.0,3.094460e+01,1.384541e+01,1.000000e+00,2.200000e+01,3.100000e+01,4.400000e+01,5.300000e+01
first_active_monthDay,201917.0,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
first_active_monthDayofweek,201917.0,3.179480e+00,1.893709e+00,0.000000e+00,2.000000e+00,3.000000e+00,5.000000e+00,6.000000e+00
first_active_monthDayofyear,201917.0,1.944820e+02,1.017959e+02,1.000000e+00,1.210000e+02,2.130000e+02,2.750000e+02,3.360000e+02


In [6]:
n = len(train_df); n

201917

## Data pipeline

In [7]:
train_df = pd.read_pickle(PATH/'joined')
test_df = pd.read_pickle(PATH/'joined_test')

In [8]:
train_df.shape, test_df.shape

((201917, 154), (123623, 152))

In [9]:
procs=[FillMissing, Categorify, Normalize]

In [10]:
dep_var = 'target'
exclude = ['first_active_month', 'card_id', 'outliers']
cont_vars,cat_vars = cont_cat_split(train_df, max_card=20, dep_var=dep_var)

In [11]:
cont_vars = list(set(cont_vars)-set(exclude))
cat_vars  = list(set(cat_vars)-set(exclude))

In [12]:
strat = StratifiedKFold(n_splits=5, shuffle=True, random_state=15)
folds = list(strat.split(train_df, train_df.outliers))

In [ ]:
#db0 = TabularList.from_df(train_df, path=PATH, cat_names=cat_vars, cont_names=cont_vars, procs=procs); db0;

In [ ]:
#db1 = db0.split_by_idx(folds[0][1]); db1;

In [ ]:
#db2 = db1.label_from_df(cols=dep_var, label_cls=FloatList)

In [ ]:
#data = db2.databunch()

In [14]:
data = (TabularList.from_df(train_df, path=PATH, cat_names=cat_vars, cont_names=cont_vars, procs=procs)
                   .split_by_idx(folds[0][1])
                   .label_from_df(cols=dep_var, label_cls=FloatList)
                   .databunch())

## Model

In [ ]:
max_log_y = np.log(np.max(train_df['Sales'])*1.2)
y_range = torch.tensor([0, max_log_y], device=defaults.device)

In [ ]:
learn = tabular_learner(data, layers=[1000,500], ps=[0.001,0.01], emb_drop=0.04, 
                        y_range=y_range, metrics=exp_rmspe)

In [ ]:
learn.model

In [ ]:
len(data.train_ds.cont_names)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 1e-3, wd=0.2)

In [ ]:
learn.save('1')

In [ ]:
learn.recorder.plot_losses(last=-1)

In [ ]:
learn.load('1');

In [ ]:
learn.fit_one_cycle(5, 3e-4)

In [ ]:
learn.fit_one_cycle(5, 3e-4)

(10th place in the competition was 0.108)

## fin